In [92]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv


In [93]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from string import digits
import nltk
import re
import string

In [94]:
lines = pd.read_csv('/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv')
lines = lines[:30000]
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [95]:
from englisttohindi.englisttohindi import EngtoHindi

msg=['Definitely share your feedback in the comment section',
     'So even if it is a big video, I will clearly mention all the products',
     ' I was waiting for my bag','I had about a 30 minute demo just using this new headset']

for i in range(len(msg)):
    x=msg[i]
    y=EngtoHindi(msg[i]).convert
    lines.english[i]=x
    lines.hindi[i]=y
lines.head()

,hindi,english
0,अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा...,Definitely share your feedback in the comment ...
1,"इसलिए भले ही यह एक बड़ा वीडियो हो, मैं स्पष्ट ...","So even if it is a big video, I will clearly m..."
2,मैं अपने बैग का इंतजार कर रहा था,I was waiting for my bag
3,मैंने इस नए हेडसेट का उपयोग करके लगभग 30 मिनट ...,I had about a 30 minute demo just using this n...
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [96]:
# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: str(x))
lines['hindi']=lines['hindi'].apply(lambda x: str(x))
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())

In [97]:
lines['hindi'][0]

'अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा करें'

In [98]:
# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub("'", '', x))

In [99]:
lines.head()

,hindi,english
0,अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा...,definitely share your feedback in the comment ...
1,"इसलिए भले ही यह एक बड़ा वीडियो हो, मैं स्पष्ट ...","so even if it is a big video, i will clearly m..."
2,मैं अपने बैग का इंतजार कर रहा था,i was waiting for my bag
3,मैंने इस नए हेडसेट का उपयोग करके लगभग 30 मिनट ...,i had about a 30 minute demo just using this n...
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,a list of plugins that are disabled by default


In [100]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [101]:
lines.head()

,hindi,english
0,अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा...,definitely share your feedback in the comment ...
1,इसलिए भले ही यह एक बड़ा वीडियो हो मैं स्पष्ट र...,so even if it is a big video i will clearly me...
2,मैं अपने बैग का इंतजार कर रहा था,i was waiting for my bag
3,मैंने इस नए हेडसेट का उपयोग करके लगभग 30 मिनट ...,i had about a 30 minute demo just using this n...
4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,a list of plugins that are disabled by default


In [102]:
remove_digits = str.maketrans('', '', digits)

In [103]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [104]:
a = lines['english'][0].translate(remove_digits)

In [105]:
a

'definitely share your feedback in the comment section'

In [106]:
a.strip()

'definitely share your feedback in the comment section'

In [107]:
# Remove all numbers from text
# remove_digits = str.maketrans('', '', digits)
# lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
# lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

# lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [108]:
'hello! how are you buddy?'.strip()

'hello! how are you buddy?'

In [109]:
lines['english'][0]

'definitely share your feedback in the comment section'

In [110]:
# Add start and end tokens to target sequences
lines['hindi'] = lines['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [111]:
lines['hindi'][0]

'START_ अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा करें _END'

In [112]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [113]:
lines.head()

,hindi,english
0,START_ अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश...,definitely share your feedback in the comment ...
1,START_ इसलिए भले ही यह एक बड़ा वीडियो हो मैं स...,so even if it is a big video i will clearly me...
2,START_ मैं अपने बैग का इंतजार कर रहा था _END,i was waiting for my bag
3,START_ मैंने इस नए हेडसेट का उपयोग करके लगभग 3...,i had about a 30 minute demo just using this n...
4,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप...,a list of plugins that are disabled by default


In [114]:
lines['length_eng']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin']=lines['hindi'].apply(lambda x:len(x.split(" ")))

In [115]:
lines.head()
lines[lines['length_eng']>30].shape

(110, 4)

In [116]:
lines=lines[lines['length_eng']<=20]
lines=lines[lines['length_hin']<=20]

In [117]:
print("maximum length of Hindi Sentence ",max(lines['length_hin']))
print("maximum length of English Sentence ",max(lines['length_eng']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [118]:
max_length_src=max(lines['length_hin'])
max_length_tar=max(lines['length_eng'])
print(max_length_src)

20


In [119]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2735, 3491)

In [120]:
num_decoder_tokens

3491

In [121]:
num_decoder_tokens += 1

In [122]:
num_decoder_tokens

3492

In [123]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [124]:
input_token_index

{'0': 1,
 '02110': 2,
 '02d': 3,
 '02i': 4,
 '1': 5,
 '10': 6,
 '100': 7,
 '1301': 8,
 '16': 9,
 '1f': 10,
 '2': 11,
 '2006': 12,
 '2007': 13,
 '2440': 14,
 '3': 15,
 '30': 16,
 '4': 17,
 '5': 18,
 '51': 19,
 '6': 20,
 '64': 21,
 '6d': 22,
 '7': 23,
 '8': 24,
 '9': 25,
 '90': 26,
 'a': 27,
 'able': 28,
 'abort': 29,
 'aborted': 30,
 'about': 31,
 'above': 32,
 'absolute': 33,
 'ac': 34,
 'accelerator': 35,
 'accelerator…': 36,
 'accentuated': 37,
 'accept': 38,
 'acceptable': 39,
 'accepted': 40,
 'accerciser': 41,
 'access': 42,
 'accessed': 43,
 'accessibility': 44,
 'accessible': 45,
 'accessibles': 46,
 'according': 47,
 'accordion': 48,
 'ace': 49,
 'aces': 50,
 'across': 51,
 'act': 52,
 'acti': 53,
 'action': 54,
 'actionable': 55,
 'actions': 56,
 'activatable': 57,
 'activate': 58,
 'activated': 59,
 'active': 60,
 'actual': 61,
 'adaptive': 62,
 'add': 63,
 'added': 64,
 'addition': 65,
 'additional': 66,
 'addrcheck': 67,
 'address': 68,
 'addressbook': 69,
 'addresses': 70,

In [125]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
reverse_input_char_index

In [127]:
lines.head(10)

,hindi,english,length_eng,length_hin
0,START_ अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश...,definitely share your feedback in the comment ...,8,10
1,START_ इसलिए भले ही यह एक बड़ा वीडियो हो मैं स...,so even if it is a big video i will clearly me...,15,19
2,START_ मैं अपने बैग का इंतजार कर रहा था _END,i was waiting for my bag,6,10
3,START_ मैंने इस नए हेडसेट का उपयोग करके लगभग 3...,i had about a 30 minute demo just using this n...,12,15
4,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप...,a list of plugins that are disabled by default,9,14
5,START_ अवधि को हाइलाइट रकें _END,highlight duration,2,6
6,START_ पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट...,the duration of the highlight box when selecti...,10,12
7,START_ सीमांत बोर्डर के रंग को हाइलाइट करें _END,highlight border color,3,9
8,START_ हाइलाइट किए गए सीमांत का रंग और अपारदर्...,the color and opacity of the highlight border,8,10
9,START_ भराई के रंग को हाइलाइट करें _END,highlight fill color,3,8


In [128]:
from sklearn.model_selection import train_test_split
X, y = lines['english'], lines['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((23712,), (5928,))

In [129]:
X_train

24167                                merged s
7189                             task manager
2100         move a onto the jack of diamonds
15131                            run continue
11560                                   force
                         ...                 
21826    field s is mandatory please enter it
5443                             six of clubs
865                                select all
15993         new project creation has failed
23930                              quick open
Name: english, Length: 23712, dtype: object

In [130]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [131]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [132]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [133]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [134]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [135]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 100

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    820800      ['input_5[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 300)    1047900     ['input_6[0][0]']                
                                                                                            

In [136]:
model.save('eng-to-hindi.h5')

In [137]:
a, b = next(generate_batch())

In [138]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [139]:
X_train[4]

'a list of plugins that are disabled by default'

In [140]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=100,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/100


/tmp/ipykernel_33/4289143106.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


370/370 [==============================] - 45s 99ms/step - loss: 5.0005 - accuracy: 0.2599 - val_loss: 4.1197 - val_accuracy: 0.3183
Epoch 2/100
370/370 [==============================] - 14s 37ms/step - loss: 3.4710 - accuracy: 0.3989 - val_loss: 2.8392 - val_accuracy: 0.4830
Epoch 3/100
370/370 [==============================] - 14s 38ms/step - loss: 2.2092 - accuracy: 0.5844 - val_loss: 1.8001 - val_accuracy: 0.6719
Epoch 4/100
370/370 [==============================] - 14s 37ms/step - loss: 1.3006 - accuracy: 0.7575 - val_loss: 1.2099 - val_accuracy: 0.7995
Epoch 5/100
370/370 [==============================] - 14s 38ms/step - loss: 0.8182 - accuracy: 0.8498 - val_loss: 0.9331 - val_accuracy: 0.8500
Epoch 6/100
370/370 [==============================] - 14s 37ms/step - loss: 0.5703 - accuracy: 0.8915 - val_loss: 0.7944 - val_accuracy: 0.8765
Epoch 7/100
370/370 [==============================] - 14s 38ms/step - loss: 0.4254 - accuracy: 0.9143 - val_loss: 0.7126 - val_accuracy: 0.88

In [141]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [142]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [143]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [144]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input English sentence: merged s
Actual Hindi Translation:  मिलाया से 
Predicted Hindi Translation:  मिलाया से 


In [145]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: task manager
Actual Hindi Translation:  फाइल का नाम ढूंढो 
Predicted Hindi Translation:  कार्य प्रबंधक 


In [146]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: move a onto the jack of diamonds
Actual Hindi Translation:  a को एक ईंट का गुलाम पर ले जाएँ 
Predicted Hindi Translation:  a को एक ईंट का गुलाम पर ले जाएँ 


In [188]:
decoded_sentence

' उपयोग के लिए खाली का उपयोग करके लगभग 30 मिनट का _END'

In [158]:
def convert_vector(input_text,max_length_src):
    # Preprocess the input text
    input_seq = np.zeros((1, max_length_src), dtype='float32')
    for t, word in enumerate(input_text.split()):
        input_seq[0, t] = input_token_index.get(word, 0)  # Use 0 for unknown words
    
    return input_seq


In [166]:
x="I had about a 30 minute demo just using this new headset"
decoded_sentence = decode_sequence(convert_vector(x,20))
print('Input English sentence:',x)
print('Predicted Hindi Translation:', decoded_sentence[:-4])

print("-"*50)
x="Definitely share your feedback in the comment section"
decoded_sentence = decode_sequence(convert_vector(x,20))
print('Input English sentence:',x)
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input English sentence: I had about a 30 minute demo just using this new headset
Predicted Hindi Translation:  उपयोग के लिए खाली का उपयोग करके लगभग 30 मिनट का 
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
Input English sentence: Definitely share your feedback in the comment section
Predicted Hindi Translation:  अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा 


In [176]:
t=[]
for m in msg:
    decoded_sentence = decode_sequence(convert_vector(m,20))
    t.append(decoded_sentence[:-4])
    

1/1 [==============================] - 0s 23ms/step


In [186]:
#using a predefine transaltor  Engtohindi
# importing the module
from englisttohindi.englisttohindi import EngtoHindi

for m,i in zip(msg,t):
    print('[English]:\t  |',m)
    print('[trans Hindi]:\t  |',EngtoHindi(m).convert)
    print('[Model predict ]: |',i)
    print('-'*50)

[English]:	  | Definitely share your feedback in the comment section
[trans Hindi]:	  | अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा करें
[Model predict ]: |  अपनी प्रतिक्रिया टिप्पणी अनुभाग में अवश्य साझा 
--------------------------------------------------
[English]:	  | So even if it is a big video, I will clearly mention all the products
[trans Hindi]:	  | इसलिए भले ही यह एक बड़ा वीडियो हो, मैं स्पष्ट रूप से सभी उत्पादों का उल्लेख करूंगा
[Model predict ]: |  इसलिए भले ही यह एक बड़ा वीडियो हो मैं स्पष्ट रू
--------------------------------------------------
[English]:	  |  I was waiting for my bag
[trans Hindi]:	  | मैं अपने बैग का इंतजार कर रहा था
[Model predict ]: |  मैं अपने बैग का इंतजार कर रहा था 
--------------------------------------------------
[English]:	  | I had about a 30 minute demo just using this new headset
[trans Hindi]:	  | मैंने इस नए हेडसेट का उपयोग करके लगभग 30 मिनट का डेमो लिया
[Model predict ]: |  उपयोग के लिए खाली का उपयोग करके लगभग 30 मिनट का 
--------------

In [148]:
# !pip install englisttohindi